## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Jardim,BR,83.88,broken clouds,-21.4803,-56.1381,Estancia Hotel - Jardim MS
1,2,Kapaa,US,71.92,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
2,4,Busselton,AU,62.10,overcast clouds,-33.6500,115.3333,Observatory Guest House
3,5,Rikitea,PF,77.94,light rain,-23.1203,-134.9692,People ThankYou
4,9,Lata,PT,70.07,clear sky,40.1629,-8.3327,Casa Maquia


In [4]:
clean_hotel_df = vacation_df

In [5]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description} </dd
</dl>

"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [10]:
# 4a. Add a marker layer for each city to the map.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [17]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[(vacation_df['City']=='Takoradi')
                                & (vacation_df['Hotel Name']=='Raybow International Hotel')]
vacation_end = vacation_df.loc[(vacation_df['City']=='Takoradi')
                              & (vacation_df['Hotel Name']=='Raybow International Hotel')]
vacation_stop1 = vacation_df.loc[(vacation_df['City']=='Begoro')
                              & (vacation_df['Hotel Name']=='Sweet Memories Hotel')]
vacation_stop2 = vacation_df.loc[(vacation_df['City']=='Ketou')
                                & (vacation_df['Hotel Name']=='Residences Celine Hotel Ketou')] 
vacation_stop3 = vacation_df.loc[(vacation_df['City']=='Ohafia')
                                & (vacation_df['Hotel Name']=='Hill Flower Hotel')]

In [18]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = pd.DataFrame(vacation_start[["Lat", "Lng"]]).to_numpy()
end = pd.DataFrame(vacation_end[["Lat", "Lng"]]).to_numpy()
stop1 = pd.DataFrame(vacation_stop1[["Lat", "Lng"]]).to_numpy()
stop2 = pd.DataFrame(vacation_stop2[["Lat", "Lng"]]).to_numpy()
stop3 = pd.DataFrame(vacation_stop3[["Lat", "Lng"]]).to_numpy()
start, stop1, stop2, stop3

(array([[ 4.8845, -1.7554]]),
 array([[ 6.3871, -0.3774]]),
 array([[7.3633, 2.5998]]),
 array([[5.6167, 7.8333]]))

In [22]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
start=(4.8845, -1.7554)
end=(4.8845, -1.7554)
stop1=(6.3871, -0.3774)
stop2=(7.3633, 2.5998)
stop3=(5.6167, 7.8333)

fig = gmaps.figure()
itinerary = gmaps.directions_layer(start,end, 
                                             waypoints=[stop1,stop2,stop3],
                                             travel_mode="DRIVING")
fig.add_layer(itinerary)
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,647,Takoradi,GH,84.40,broken clouds,4.8845,-1.7554,Raybow International Hotel
1,351,Begoro,GH,92.01,overcast clouds,6.3871,-0.3774,Sweet Memories Hotel
2,242,Ketou,BJ,99.88,broken clouds,7.3633,2.5998,Residences Celine Hotel Ketou
3,565,Ohafia,NG,98.94,few clouds,5.6167,7.8333,Hill Flower Hotel


In [24]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map.
itinerary_locations = itinerary_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(36.0, -76.0), zoom_level=3)
marker_layer = gmaps.marker_layer(itinerary_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))